# Loading acquisition parameters and setting up an acquisition scheme

dmipy saves all acquisition parameter information inside an DmipyAcquisitionScheme instance. Acquisition schemes can be set up directly from b-values, q-values or gradient strength; can be converted from a dipy GradientTable; or be imported from a Camino-style schemefile.

## Setting up a Dmipy acquisition scheme by loading acquisition parameters

To set up an acquisition scheme directly from b-values one can load them directly from text files. The b-values and gradient directions, along with the used pulse duration time $\delta$ and pulse separation time $\Delta$ of the acquisition must be known. The dmipy toolbox uses SI units, so be careful, as bvalues are typically saved in $s/mm^2$, but as a rule we need them in $s/m^2$.

As an example we load the acquisition parameters of the WU-MINN Human Connectome Project:

In [1]:
# load the necessary modules
from dmipy.core import modeling_framework
from dmipy.core.acquisition_scheme import acquisition_scheme_from_bvalues
from os.path import join
import numpy as np

# the HCP acquisition parameters are saved in the following toolbox path:
acquisition_path = modeling_framework.GRADIENT_TABLES_PATH

# we can then load the parameters themselves and convert them to SI units:
bvalues = np.loadtxt(join(acquisition_path, 'bvals_hcp_wu_minn.txt'))  # given in s/mm^2
bvalues_SI = bvalues * 1e6  # now given in SI units as s/m^2
gradient_directions = np.loadtxt(join(acquisition_path, 'bvecs_hcp_wu_minn.txt'))  # on the unit sphere

# The delta and Delta times we know from the HCP documentation in seconds
delta = 0.0106  
Delta = 0.0431 

# The acquisition scheme we use in the toolbox is then created as follows:
acq_scheme = acquisition_scheme_from_bvalues(bvalues_SI, gradient_directions, delta, Delta)

From the acquisition scheme, some basic information can be called using the following command:

In [2]:
acq_scheme.print_acquisition_info

Acquisition scheme summary

total number of measurements: 288
number of b0 measurements: 18
number of DWI shells: 3

shell_index |# of DWIs |bvalue [s/mm^2] |gradient strength [mT/m] |delta [ms] |Delta[ms] |TE[ms]
0           |18        |0               |0                        |10.6       |43.1      |N/A  
1           |90        |1000            |56                       |10.6       |43.1      |N/A  
2           |90        |2000            |79                       |10.6       |43.1      |N/A  
3           |90        |3000            |97                       |10.6       |43.1      |N/A  


As the summary shows, the function automatically separates the different acquisition shells and detects b0 measurements.  
Other metrics such as qvalues, gradient strengths, diffusion times and the shell_indices can also be explicitly called as follows:

In [3]:
acq_scheme.bvalues;  # bvalues in s/m^2
acq_scheme.gradient_directions;  # gradient directions on the unit sphere
acq_scheme.gradient_strengths;  # the gradient strength in T/m
acq_scheme.qvalues;  # describes the diffusion sensitization in 1/m
acq_scheme.tau;  # diffusion time as Delta - delta / 3. in seconds
acq_scheme.shell_indices;  # index assigned to each shell. 0 is assigned to b0 measurements

## Setting up a Dmipy acquisition scheme by converting a Dipy GradientTable

Dipy's GradientTable and Dmipy's AcquisitionScheme both store and verify a dataset's acquisition parameters to later be used in model fitting. For convenience, we provide the possibility to convert a dipy table to a dmipy scheme using a conversion function.

In [4]:
from dipy.core.gradients import gradient_table
from dmipy.core.acquisition_scheme import gtab_dipy2dmipy
gtab_dipy = gradient_table(bvalues, gradient_directions, big_delta=Delta, small_delta=delta)
acq_scheme_mipy = gtab_dipy2dmipy(gtab_dipy)
acq_scheme_mipy.print_acquisition_info

Acquisition scheme summary

total number of measurements: 288
number of b0 measurements: 18
number of DWI shells: 3

shell_index |# of DWIs |bvalue [s/mm^2] |gradient strength [mT/m] |delta [ms] |Delta[ms] |TE[ms]
0           |18        |0               |0                        |10.6       |43.1      |N/A  
1           |90        |1000            |56                       |10.6       |43.1      |N/A  
2           |90        |2000            |79                       |10.6       |43.1      |N/A  
3           |90        |3000            |97                       |10.6       |43.1      |N/A  


As you can see, the result is exactly the same. Note that the conversion requires that big_delta and small_delta are both set in the gradient_table!

## Setting up a Dmipy acquisition scheme from a Camino schemefile

It is also possible to create a dmipy acquisition scheme directly from a Camino schemefile. Below we load a Camino schemefile containing the same wu-minn HCP acquisition parameters directly from the file itself.

In [5]:
from dmipy.core.acquisition_scheme import acquisition_scheme_from_schemefile
acq_scheme_mipy = acquisition_scheme_from_schemefile(
    join(acquisition_path, "schemefile_hcp_wu_minn.scheme1")
)
acq_scheme_mipy.print_acquisition_info

Acquisition scheme summary

total number of measurements: 288
number of b0 measurements: 18
number of DWI shells: 3

shell_index |# of DWIs |bvalue [s/mm^2] |gradient strength [mT/m] |delta [ms] |Delta[ms] |TE[ms]
0           |18        |0               |0                        |10.6       |43.1      |65.3 
1           |90        |1000            |56                       |10.6       |43.1      |65.3 
2           |90        |2000            |79                       |10.6       |43.1      |65.3 
3           |90        |3000            |97                       |10.6       |43.1      |65.3 


It is also possible so save an AcquisitionScheme to a schemefile file as follows

In [6]:
acq_scheme_mipy.to_schemefile(join(acquisition_path, "schemefile_hcp_wu_minn.scheme1"))

If the acquisition scheme has no TE indicated, then in the schemefile it will default to TE=Delta + 2delta + 0.001 seconds